In [ ]:
!pip install skforecast

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import scatter_matrix
import missingno as msno
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [ ]:
# Descarga de datos
# ======================
file = ('https://raw.githubusercontent.com/JorgeMendiProject/TFM/main/DatasetInterpolado.csv')

# Lectura de datos a partir del archivo .csv             

df = pd.read_csv(file, delimiter=';')
df.info()

In [3]:
# Convertimos la variable Fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')
df = df.set_index('Fecha')

#Creamos las variables con año y mes para analizar las series temporales

df['Year'] = df.index.year
df['Month'] = df.index.month

In [4]:
list_to_plot = ['TasaTPIB','EuriborM',
       'ParoT', 'SalarioMedio', 'IPC', 'CTotalConsumo']   

In [ ]:
# P- valor
# la prueba ADF sirve para comprobar si existe estacionalidad en las variables, 
# Cuando el estadístico de prueba es menor que el valor crítico mostrado, rechaza la hipótesis nula e infiere que la serie de tiempo es estacional.
# Si adf es menor que el P-valor se infiere que hay estacionalidad

from statsmodels.tsa.stattools import adfuller
for i in list_to_plot:
  X = df[i].values
  X = np.nan_to_num(X)
  result = adfuller(X)
  print(i), print('ADF Statistic: %f' % result[0]), print('p-value: %f' % result[1]), print('Critical Values:')
  for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


In [ ]:
# Factores autorregresivos de las variables, se puede interpretar que las variables TasaPIB, EuriborM, ParoT, SalarioMedio,
# tiene un factor autorregresivo lo que implica que lo que esta pasando en el momento T actual,
# tiene que ver con lo que el valor que alcanzó la variable en el momento anterior.
# Sin embargo en el caso del IPC existe una mayor dispersión y parece que no existe esta relación.

from pandas.plotting import lag_plot
for i in list_to_plot:
  print(i)
  lag_plot(df[i])
  plt.show()

In [7]:
# Primera diferenciación

df_diff = df.diff()
df_diff['Year'] = df['Year']
df_diff['Month'] = df['Month']

In [ ]:
# P- valor
# la prueba ADF sirve para comprobar si existe estacionalidad en las variables,
# Cuando el estadístico de prueba es menor que el valor crítico mostrado, rechaza la hipótesis nula e infiere que la serie de tiempo es estacional.
# Si adf es menor que el P-valor se infiere que hay estacionalidad

from statsmodels.tsa.stattools import adfuller
for i in list_to_plot:
  X = df_diff[i].values
  X = np.nan_to_num(X)
  result = adfuller(X)
  print(i), print('ADF Statistic: %f' % result[0]), print('p-value: %f' % result[1]), print('Critical Values:')
  for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
# Factores autorregresivos de las variables, se puede interpretar que las variables TasaPIB, EuriborM, ParoT, SalarioMedio 
# tiene un factor autorregresivo lo que implica que lo que esta pasando en el momento T actual,
# tiene que ver con lo que el valor que alcanzó la variable en el momento anterior.
#Sin embargo en el caso del IPC existe una mayor dispersión y parece que no existe esta relación.

from pandas.plotting import lag_plot
for i in list_to_plot:
  print(i)
  lag_plot(df_diff[i])
  plt.show()

In [10]:
# Segunda diferenciación

df_diff_diff = df.diff().diff()
df_diff_diff['Year'] = df['Year']
df_diff_diff['Month'] = df['Month']

In [ ]:
# P- valor
# la prueba ADF sirve para comprobar si existe estacionalidad en las variables, Cuando el estadístico de prueba es menor que el valor crítico mostrado,
# rechaza la hipótesis nula e infiere que la serie de tiempo es estacional. Si adf es menor que el P-valor se infiere que hay estacionalidad

from statsmodels.tsa.stattools import adfuller
for i in list_to_plot:
  X = df_diff_diff[i].values
  X = np.nan_to_num(X)
  result = adfuller(X)
  print(i), print('ADF Statistic: %f' % result[0]), print('p-value: %f' % result[1]), print('Critical Values:')
  for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
# Factores autorregresivos de las variables, se puede interpretar que las variables TasaPIB, EuriborM, ParoT, SalarioMedio,
# tiene un factor autorregresivo lo que implica que lo que esta pasando en el momento T actual,
# tiene que ver con lo que el valor que alcanzó la variable en el momento anterior.
# Sin embargo en el caso del IPC existe una mayor dispersión y parece que no existe esta relación.

from pandas.plotting import lag_plot
for i in list_to_plot:
  print(i)
  lag_plot(df_diff_diff[i])
  plt.show()

In [ ]:
# Descomposición de series temporales

from statsmodels.tsa.seasonal import STL
for i in list_to_plot:
  plt.rc('figure',figsize=(16,12))
  plt.rc('font',size=10)
  Y = df[i].fillna(0)
  stl = STL(Y)
  res = stl.fit()
  fig = res.plot()

In [ ]:
# Descomposición de series temporales

from statsmodels.tsa.seasonal import STL
for i in list_to_plot:
  plt.rc('figure',figsize=(16,12))
  plt.rc('font',size=10)
  Y = df_diff[i].fillna(0)
  stl = STL(Y)
  res = stl.fit()
  fig = res.plot()

In [ ]:
# define list_to_plot
list_to_plot = ['TasaTPIB','EuriborM','IPC']  

# define order_model
order_model = [(1,1,1), (1,1,2), (2,1,1)]

# Ejecuta todos los posibles modelos del order model, para todas las variables y devuelve el Modelo que mejor se ajusta

best_model = {}
model_results = []
for i in list_to_plot:
  counter = 0
  previous_best_aic_model = 0
  for j in order_model:
    # fit model
    model = sm.tsa.arima.ARIMA(df_diff[i].dropna(), order=j, dates=np.array(df_diff.reset_index().Fecha[1:]), freq='M')
    model_fit = model.fit()
    model_results.append(("TasaTPIB", i, " Model AIC ", model_fit.aic, " Model Specification ", j))
    if counter == 0:
      previous_best_aic_model = model_fit.aic
      best_model[i] = j
      counter +=1
    else:
      if previous_best_aic_model > model_fit.aic:
        previous_best_aic_model = model_fit.aic
        best_model[i] = j
        counter +=1
      else:
        counter+=1
    # summary of fit model
    print(model_fit.summary())
    # line plot of residuals
    residuals = pd.DataFrame(model_fit.resid) 
    residuals.plot()
    plt.show()
    # density plot of residuals 
    residuals.plot(kind='kde') 
    plt.show()
    # summary stats of residuals
    print(residuals.describe())


In [ ]:
# Se imprime el mejor modelo para cada variable

best_model

In [ ]:
# Imprime un resumen con el mejor modelo ARIMA para cada variable

for key in best_model:
  model = sm.tsa.arima.ARIMA(df_diff[key].dropna(), order=best_model[key])
  model_fit = model.fit()
  # print(model_fit.aic, i, j)
  # summary of fit model
  print(model_fit.summary())
  # line plot of residuals
  residuals = pd.DataFrame(model_fit.resid) 
  residuals.plot()
  plt.show()
  # density plot of residuals 
  residuals.plot(kind='kde') 
  plt.show()
  # summary stats of residuals
  print(residuals.describe())

In [ ]:
# Modelado para predicción con Df.diff

# {'TasaTPIB': (1, 1, 1), 'EuriborM': (1, 1, 2), 'IPC': (1, 1, 2)}

PIB_model = sm.tsa.arima.ARIMA(df_diff['TasaTPIB'].dropna(), order=(1, 1, 1 ))
PIB_fit = PIB_model.fit()
Euribor_model = sm.tsa.arima.ARIMA(df_diff['EuriborM'].dropna(), order=(1, 1, 2))
Euribor_fit = Euribor_model.fit()
IPC_model = sm.tsa.arima.ARIMA(df_diff['IPC'].dropna(), order=(1, 1, 2))
IPC_fit = IPC_model.fit()


In [ ]:
#PIB model
PIB_model_impulse = PIB_model.impulse_responses(params=PIB_fit.params, steps=12, cumulative=False)
PIB_model_impulse.plot()
     
#Euribor model
Euribor_model_impulse = Euribor_model.impulse_responses(params=Euribor_fit.params, steps=12, cumulative=False)
Euribor_model_impulse.plot()

#IPC model
IPC_model_impulse = IPC_model.impulse_responses(params=IPC_fit.params, steps=12, cumulative=False)
IPC_model_impulse.plot()


In [ ]:
PIB_model = ARIMA(df_diff['TasaTPIB'].dropna(), order=(1, 1, 2))
PIB_fit = PIB_model.fit()
print(len(df_diff))

end = len(df)
PIB_model_impulse = PIB_fit.plot_predict(end=end)

m_train = 12

df_train = df[:-m_train] 
df_test = df[-m_train:]

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para test: ", len(df_test))

df_test['TasaTPIB'].dropna().plot() #azúl

df_test['TasaTPIB'].diff().dropna().plot() #Naranja

pred, stderr, conf_int = PIB_fit.forecast(steps=12)
print("Intervalo de predicción: [{}, {}]".format(conf_int[0][0], conf_int[0][1]))

y_true = df_test['TasaTPIB'].dropna()
y_pred = pred

mse = mean_squared_error(y_true, y_pred)
print("MSE: ", mse)
